In [9]:
from langchain_community.document_loaders import DirectoryLoader

In [14]:
loader = DirectoryLoader("/Users/hritik/Desktop/Grading Doc RAG/Document", glob="**/*.html")
docs = loader.load()
len(docs)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hritik/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


166

In [15]:
docs

[Document(page_content='English\n\n| हिंदी\n\n| தமிழ்\n\n| తెలుగు\n\n| മലയാളം\n\n| ಕನ್ನಡ\n\n| मराठी\n\n| ગુજરાતી\n\n| বাংলা\n\nTags for Study at IITM\n\nContact Us\n\n\r\n                Indian Institute of Technology Madras \n\nIIT P.O., Chennai 600 036\n\nINDIA\n\nView Site Map\n\nView Site Map\n\nAbout IITM\n\nPrivacy Policy\r\n              |\n\nTerms of Service\r\n              |\n\nAccessibility', metadata={'source': '/Users/hritik/Desktop/Grading Doc RAG/Document/www/tags.html'}),
 Document(page_content="English\n\n| हिंदी\n\n| தமிழ்\n\n| తెలుగు\n\n| മലയാളം\n\n| ಕನ್ನಡ\n\n| मराठी\n\n| ગુજરાતી\n\n| বাংলা\n\nShare this page with your friends and family on\n\nOn-Campus Programs\n\nNon-Campus Undergraduate Programs\n\nBS in Data Science\n\nBS in Data Science\n\nand Application\n\nBS in Electronic Systems\n\nDiploma in Programming\n\nDiploma in Programming\n\n/Data Science\n\nUndergraduate\n\nUndergraduate\n\nProgrammes\n\nPostgraduate\n\nPostgraduate\n\nProgrammes\n\nResearch Program